In [1]:
import random
import re
import pickle

import openai

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether
from scapy.layers.inet import IP, TCP, ICMP

In [2]:
packets_per_request = 10

In [3]:
with open("../data/text/three_summaries.txt","r") as f:
    packets_summary = f.read().splitlines()

In [4]:
len(packets_summary)

9999

In [40]:
engines = openai.Engine.list()


In [6]:
responses = pickle.load( open("../data/pickle/davinci_pairs.pkl", "rb" ))


In [8]:
len(responses)

2

In [165]:
summaries = random.choices(packets_summary,k=packets_per_request)
query = "These are the packet summaries:\n"
text_sum = ""
for summary in summaries:
    query += summary + "\n"
    text_sum += summary + "\n"
    
query += "\nGenerate python code for creating these packets with Scapy framework and put them all in a list named pkt_list."
    
print(query)
    
completion = openai.Completion.create(engine="text-davinci-003", prompt=query,max_tokens=2600,temperature=0.1)
completion["prompt_summary"] = text_sum
responses.append(completion)

print(responses[-1].choices[0].text)

These are the packet summaries:
192.168.1.190 → 192.168.1.194 ICMP 100 Echo (ping) reply id=0x9015, seq=19497
192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=19490
192.168.1.193 → 192.168.1.190 ICMP 76 Echo (ping) request id=0x0001, seq=4723
192.168.1.190 → 192.168.1.152 DNS 193 Standard query response 0x0971 No such name AAAA a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com SOA ns-1926.awsdns-48.co.uk
192.168.1.193 → 192.168.1.190 ICMP 76 Echo (ping) request id=0x0001, seq=5007
192.168.1.190 → 192.168.1.152 DNS 193 Standard query response 0x063b No such name AAAA a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com SOA ns-1926.awsdns-48.co.uk
192.168.1.192 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x112d, seq=5550
192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=19685
192.168.1.190 → 192.168.1.194 ICMP 100 Echo (ping) reply id=0x9015, seq=18947
192.168.1.190 → 192.168.1.192 ICMP 100 Echo (ping) reply id=0x112d, seq=5643

Generate pyth

In [166]:
print(responses[-1].choices[0].text)



pkt_list = []

pkt1 = IP(src="192.168.1.190", dst="192.168.1.194")/ICMP(type="echo-reply", id=0x9015, seq=19497)
pkt_list.append(pkt1)

pkt2 = IP(src="192.168.1.194", dst="192.168.1.190")/ICMP(type="echo-request", id=0x9015, seq=19490)
pkt_list.append(pkt2)

pkt3 = IP(src="192.168.1.193", dst="192.168.1.190")/ICMP(type="echo-request", id=0x0001, seq=4723)
pkt_list.append(pkt3)

pkt4 = IP(src="192.168.1.190", dst="192.168.1.152")/DNS(id=0x0971, qr=1, opcode=0, rcode=3, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="SOA", qclass="IN"))
pkt_list.append(pkt4)

pkt5 = IP(src="192.168.1.193", dst="192.168.1.190")/ICMP(type="echo-request", id=0x0001, seq=5007)
pkt_list.append(pkt5)

pkt6 = IP(src="192.168.1.190", dst="192.168.1.152")/DNS(id=0x063b, qr=1, opcode=0, rcode=3, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="SOA", qclass="IN"))
pkt_list.appen

In [167]:
exec(responses[-1].choices[0].text)

In [168]:
pkt_list

[<IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.194 |<ICMP  type=echo-reply id=0x9015 seq=0x4c29 |>>,
 <IP  frag=0 proto=icmp src=192.168.1.194 dst=192.168.1.190 |<ICMP  type=echo-request id=0x9015 seq=0x4c22 |>>,
 <IP  frag=0 proto=icmp src=192.168.1.193 dst=192.168.1.190 |<ICMP  type=echo-request id=0x1 seq=0x1273 |>>,
 <IP  src=192.168.1.190 dst=192.168.1.152 |<DNS  id=2417 qr=1 opcode=QUERY rcode=name-error qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com.' qtype=SOA qclass=IN |> |>>,
 <IP  frag=0 proto=icmp src=192.168.1.193 dst=192.168.1.190 |<ICMP  type=echo-request id=0x1 seq=0x138f |>>,
 <IP  src=192.168.1.190 dst=192.168.1.152 |<DNS  id=1595 qr=1 opcode=QUERY rcode=name-error qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com.' qtype=SOA qclass=IN |> |>>,
 <IP  frag=0 proto=icmp src=192.168.1.192 dst=192.168.1.190 |<ICMP  type=echo-request id=0x112d seq=0x15

In [171]:
len(responses)

20

In [172]:
pickle.dump(responses, open( "../data/pickle/davinci_pairs.pkl", "wb" ) )

In [146]:
fixed_text = """


pkt_list = [
    scapy.Ether(src="192.168.1.190", dst="192.168.1.193")/scapy.IP(src="192.168.1.190", dst="192.168.1.193")/scapy.ICMP(type="echo-reply", id=0x0001, seq=5396),
    scapy.Ether(src="192.168.1.190", dst="192.168.1.194")/scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type="echo-reply", id=0x9015, seq=18669),
    scapy.Ether(src="192.168.1.193", dst="192.168.1.190")/scapy.IP(src="192.168.1.193", dst="192.168.1.190")/scapy.ICMP(type="echo-request", id=0x0001, seq=4368),
    scapy.Ether(src="192.168.1.190", dst="192.168.1.194")/scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type="echo-reply", id=0x9015, seq=19807),
    scapy.Ether(src="192.168.1.190", dst="192.168.1.193")/scapy.IP(src="192.168.1.190", dst="192.168.1.193")/scapy.ICMP(type="echo-reply", id=0x0001, seq=4797),
    scapy.Ether(src="192.168.1.190", dst="192.168.1.194")/scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type="echo-reply", id=0x9015, seq=18829),
    scapy.Ether(src="192.168.1.194", dst="192.168.1.190")/scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type="echo-request", id=0x9015, seq=19744),
    scapy.Ether(src="192.168.1.190", dst="192.168.1.193")/scapy.IP(src="192.168.1.190", dst="192.168.1.193")/scapy.ICMP(type="echo-reply", id=0x0001, seq=5266),
    scapy.Ether(src="192.168.1.194", dst="192.168.1.190")/scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type="echo-request", id=0x9015, seq=18848)
]
"""



In [147]:
responses[-1].choices[0].text = fixed_text

In [99]:
print(responses[-1].choices[0].text)




pkt_list = [
    scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x9015, seq=18455),
    scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=0, id=0x9015, seq=18820),
    scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=0, id=0x9015, seq=18566),
    scapy.IP(src="192.5.6.30", dst="192.168.1.190")/scapy.DNS(id=0x176b, qd=scapy.DNSQR(qname="google.com", qtype="NSEC3", qclass="RRSIG", rd=1, ar=scapy.DNSRR(rrname="a.gtld-servers.net", type="RRSIG", rclass="NSEC3", ttl=0, rdlen=0, rdata="OPT")), an=scapy.DNSRR(rrname="google.com", type="SOA", rclass="a.gtld-servers.net", ttl=0, rdlen=0, rdata="RRSIG")),
    scapy.IP(src="192.168.1.193", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x0001, seq=5521),
    scapy.IP(src="192.168.1.190", dst="192.168.1.192")/scapy.ICMP(type=0, id=0x112d, seq=6590),
    scapy.IP(src="192.168.1.190", dst="192.168.1.192")/scapy.ICMP(type=0, id=0x112d, seq=5868),
    scapy.IP(src="192.168.1.192", dst="192.16

In [170]:
del responses[-2]